# Item-to-Item Collaborative Filtering


## Conceito
Seja $u$ o user ativo e $i$ o item alvo.

- Se o user $u$ demonstrou preferência por itens similares ao $i$, é provável que $u$ também goste de $i$.
- Por outro lado, se $u$ não gostou de itens similares ao $i$, é provável que também não goste de $i$.

Em resumo, ao analisar como o user $u$ avaliou itens semelhantes ao $i$, podemos estimar como $u$ avaliaria o item $i$.



## Algoritmo: Filtragem Colaborativa Item-a-Item

O algoritmo de filtragem colaborativa baseada em itens pode ser descrito conforme apresentado em <a href="http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.449.1171&rep=rep1&type=pdf">(B. Sarwar et al. 2001)</a> e <a href="https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.554.1671&rep=rep1&type=pdf">(George Karypis 2001)</a>:

1. **Cálculo das Similaridades entre Itens:**  
    Para cada item do catálogo, identificam-se os $k$ itens mais similares, registrando os valores de similaridade correspondentes. A similaridade entre dois itens pode ser calculada utilizando a *Similaridade Cosseno Ajustada*, que é mais eficaz do que a similaridade cosseno tradicional usada em filtragem colaborativa baseada em users. A fórmula é:

    $$
    w_{i,j}= \frac{\sum_{u\in U}(r_{u,i}-\bar{r}_u)(r_{u,j}-\bar{r}_u)}{\sqrt{\sum_{u\in U} (r_{u,i}-\bar{r}_u)^2}\sqrt{\sum_{u\in U} (r_{u,j}-\bar{r}_u)^2}}
    $$

    Onde $w_{i,j}$ representa o grau de similaridade entre os itens $i$ e $j$, considerando todos os users $u \in U$ que avaliaram ambos os itens. $S^{(i)}$ representa o conjunto dos $k$ itens mais similares ao item $i$.

2. **Geração de Recomendações Top-N para o Usuário:**  
    Para recomendar itens a um usuário $u$ que já interagiu com um conjunto $I_u$ de itens:

    - **Seleção de Candidatos:**  
      Encontra-se o conjunto $C$ de itens candidatos, que é a união dos conjuntos $S^{(i)}$ para todos os itens $i \in I_u$, excluindo os itens já consumidos:

      $$
      C = \bigcup_{i\in I_u}\{S^{(i)}\}\smallsetminus I_u
      $$

    - **Cálculo da Similaridade Total:**  
      Para cada candidato $c \in C$, calcula-se a soma das similaridades entre $c$ e todos os itens do conjunto $I_u$:

      $$
      w_{c,I_u} = \sum_{i\in I_u} w_{c,i}, \forall c \in C
      $$

    - **Ordenação e Seleção dos Itens:**  
      Os itens candidatos são ordenados de acordo com $w_{c,I_u}$ em ordem decrescente, e os $N$ primeiros são recomendados ao usuário.

Este método permite recomendar itens relevantes ao usuário com base nas similaridades entre os itens que ele já avaliou ou consumiu.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, collect_list, struct, sum as sql_sum, udf, lit
from pyspark.ml.linalg import Vectors, SparseVector
from pyspark.sql import functions as F
from pyspark.sql import Row
from pyspark.ml.feature import Normalizer, BucketedRandomProjectionLSH
from pyspark.ml.evaluation import RegressionEvaluator
import os
import time
import pandas as pd

## Importação das Bibliotecas

O código a seguir importa as bibliotecas essenciais para o processamento distribuído de dados, manipulação de DataFrames, criação de vetores esparsos, normalização, aplicação de LSH (Locality Sensitive Hashing) e avaliação do modelo. Utilizamos o PySpark para processamento em larga escala, além de pandas para sumarização dos resultados.

In [2]:
start_time = time.time()

## Medição do Tempo de Execução

O código abaixo inicializa um temporizador para medir o tempo total de execução do pipeline de recomendação. Isso é útil para avaliar o desempenho do algoritmo em diferentes conjuntos de dados.

In [3]:
spark = SparkSession.builder \
    .appName("ItemItemCF") \
    .config("spark.driver.memory", "16g") \
    .config("spark.executor.memory", "16g") \
    .config("spark.executor.cores", "4") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .getOrCreate()

# Set logging level
spark.sparkContext.setLogLevel("WARN")

25/05/30 23:57:49 WARN Utils: Your hostname, cristianonicolau.local resolves to a loopback address: 127.0.0.1; using 192.168.1.122 instead (on interface en0)
25/05/30 23:57:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/30 23:57:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/05/30 23:57:49 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


## Inicialização da SparkSession

Aqui criamos uma sessão Spark, configurando a quantidade de memória e núcleos de execução. A SparkSession é o ponto de entrada para utilizar o PySpark e permite o processamento distribuído dos dados. Também ajustamos o nível de log para evitar excesso de mensagens durante a execução.

In [4]:
df ={
    "small": "./data/ml-latest-small/ratings.csv",
    "ml-1m": "./data/ml-1m/ratings.csv",
    "ml-10m": "./data/ml-10m/ratings.csv",
    "ml-20m": "./data/ml-20m/ratings.csv",
    "ml-25m": "./data/ml-25m/ratings.csv",
}

data_sel = "small" 

## Seleção do Dataset

Definimos um dicionário com os caminhos para diferentes versões do MovieLens. O dataset selecionado será utilizado para treinar e avaliar o modelo de recomendação.

In [5]:
# dataset_links = {
#     "small": "https://files.grouplens.org/datasets/movielens/ml-latest-small.zip",
#     "ml-1m": "https://files.grouplens.org/datasets/movielens/ml-1m.zip",
#     "ml-10m": "https://files.grouplens.org/datasets/movielens/ml-10m.zip",
#     "ml-20m": "https://files.grouplens.org/datasets/movielens/ml-20m.zip",
#     "ml-25m": "https://files.grouplens.org/datasets/movielens/ml-25m.zip"
# }

# def download_and_extract_dataset(dataset_name):
#     if dataset_name not in dataset_links:
#         raise ValueError(f"Dataset '{dataset_name}' not found. Available datasets: {list(dataset_links.keys())}")

#     url = dataset_links[dataset_name]
#     os.system(f"wget {url} -O {dataset_name}.zip")
#     os.system(f"unzip {dataset_name}.zip -d {dataset_name}")
#     print(f"Downloaded and extracted {dataset_name} dataset.")

## (Opcional) Download Automático dos Datasets

O bloco abaixo, atualmente comentado, permite baixar e extrair automaticamente os datasets do MovieLens a partir dos links oficiais. Isso facilita a obtenção dos dados necessários para os experimentos.

In [6]:
PATH = df[data_sel]

# download_and_extract_dataset("ml-25m")  # Change to desired dataset
# PATH = "ml-25m/ratings.csv"  # Adjust path based on the dataset
data = spark.read.csv(PATH, header=True, inferSchema=True) \
            .select("userId", "movieId", "rating")

print("=== DATASET STATISTICS ===")
print("Number of ratings: ", data.count())
print("Average rating: ", data.agg(F.avg("rating")).first()[0])
print("Minimum rating: ", data.agg(F.min("rating")).first()[0])
print("Maximum rating: ", data.agg(F.max("rating")).first()[0])
print("Number of users: ", data.select("userId").distinct().count())
print("Number of movies: ", data.select("movieId").distinct().count())

data.take(5)

=== DATASET STATISTICS ===
Number of ratings:  100836
Average rating:  3.501556983616962
Minimum rating:  0.5
Maximum rating:  5.0
Number of users:  610
Number of movies:  9724


[Row(userId=1, movieId=1, rating=4.0),
 Row(userId=1, movieId=3, rating=4.0),
 Row(userId=1, movieId=6, rating=4.0),
 Row(userId=1, movieId=47, rating=5.0),
 Row(userId=1, movieId=50, rating=5.0)]

## Carregamento e Estatísticas do Dataset

O código a seguir carrega o arquivo de ratings selecionado, exibe estatísticas como número de avaliações, média, mínimo, máximo, número de usuários e de filmes. Isso fornece uma visão geral do conjunto de dados utilizado.

In [7]:
# Split data into training and test sets
ratings, test = data.randomSplit([0.9, 0.1], seed=42)
print(f"Training set size: {ratings.count()}")
print(f"Test set size: {test.count()}")
ratings.cache()
test.cache()

Training set size: 90673
Test set size: 10163


DataFrame[userId: int, movieId: int, rating: double]

## Divisão em Conjuntos de Treino e Teste

Aqui dividimos o dataset em conjuntos de treino (90%) e teste (10%) de forma aleatória. O conjunto de treino é usado para construir o modelo, enquanto o teste serve para avaliar a qualidade das recomendações.

## 3. Model Training: Item-Item CF with LSH
 
The training process involves several steps:
1.  **Create Item Vectors**: For each movie, create a vector where each dimension represents a user, and the value is the rating given by that user. We use sparse vectors for efficiency.
2.  **Normalize Vectors**: Normalize the item vectors using the L2 norm. This makes the LSH approach (based on Euclidean distance) approximate cosine similarity.
3.  **Apply LSH**: Use `BucketedRandomProjectionLSH` to hash the normalized vectors into buckets. Items falling into the same buckets are likely to be similar.
4.  **Find Similar Items**: Use `approxSimilarityJoin` to find pairs of similar items based on their LSH hashes and calculate their cosine similarity.


In [8]:
def to_sparse_vector(user_ratings, size):
    # Sort by userId to ensure strictly increasing indices
    sorted_pairs = sorted(user_ratings, key=lambda x: x.userId)
    indices = [x.userId - 1 for x in sorted_pairs] # Assuming userIds start at 1
    values = [float(x.rating) for x in sorted_pairs]
    return Vectors.sparse(size, indices, values)

### 3.1 Criação dos Vetores dos Itens

A função abaixo transforma as avaliações dos usuários em vetores esparsos, onde cada dimensão representa um usuário e o valor é a nota atribuída ao item. Isso é fundamental para calcular similaridades entre itens de forma eficiente.

now that each rating has been normalized, we can represent each item by a vector of its normalized ratings

In [9]:
spark.udf.register("to_sparse_vector", to_sparse_vector)

item_user = ratings.groupBy("movieId") \
    .agg(collect_list(struct("userId", "rating")).alias("user_ratings"))

### Agrupamento das Avaliações por Item

Registramos a função de conversão para vetor esparso como UDF no Spark e agrupamos as avaliações por `movieId`, preparando os dados para a criação dos vetores de características dos itens.

In [10]:
num_users = ratings.select("userId").distinct().count()

item_vectors_rdd = item_user.rdd.map(
    lambda row: Row(
        movieId=row["movieId"],
        features=to_sparse_vector(row["user_ratings"], num_users)
    )
)
item_vectors = spark.createDataFrame(item_vectors_rdd)
item_vectors.cache()
print("=== ITEM VECTORS ===")
item_vectors.show(5, truncate=False)

=== ITEM VECTORS ===


+-------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

### Criação dos Vetores de Características dos Itens

Aqui, para cada item, criamos um vetor esparso representando as avaliações dos usuários. Esses vetores serão usados para calcular similaridades entre itens.

### 3.2 Normalize Vectors
 
We normalize the vectors so that Euclidean distance can approximate cosine similarity.


In [11]:
normalizer = Normalizer(inputCol="features", outputCol="norm_features", p=2.0)
normalized_item_vectors = normalizer.transform(item_vectors)
normalized_item_vectors.cache() # Cache for LSH

print("Normalized Item Vectors (sample):")
normalized_item_vectors.show(5, truncate=False)

Normalized Item Vectors (sample):
+-------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

### 3.2 Normalização dos Vetores

Normalizamos os vetores dos itens utilizando a norma L2. Isso permite que a distância Euclidiana aproxime a similaridade cosseno, facilitando o uso do LSH para encontrar itens similares.

## 3.3 Apply LSH

We fit an LSH model. The `bucketLength` and `numHashTables` are key parameters:
* `bucketLength`: Affects the width of the buckets. Smaller values lead to more, smaller buckets (higher precision, lower recall).
* `numHashTables`: Increases the chance of finding neighbors by using multiple hash functions (higher recall, more computation).

These parameters often require tuning based on the dataset and desired trade-off.


In [12]:
bucket_length = 1.5
num_hash_tables = 8


lsh = BucketedRandomProjectionLSH(
    inputCol="norm_features",
    outputCol="hashes",
    bucketLength= bucket_length,
    numHashTables= num_hash_tables
)

lsh_model = lsh.fit(normalized_item_vectors)

25/05/30 23:57:56 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


### 3.3 Configuração e Treinamento do LSH

Definimos os parâmetros do LSH (comprimento do bucket e número de tabelas de hash) e treinamos o modelo para agrupar itens similares em buckets, acelerando a busca por vizinhos próximos.

### 3.4 Find Similar Items
We use `approxSimilarityJoin` to find pairs with a Euclidean distance below a certain `threshold`. We then convert this distance to cosine similarity using the formula: $\cos(\theta) = 1 - \frac{d^2}{2}$. We ensure we have both (i, j) and (j, i) pairs.


In [13]:
# Set a threshold for similarity (Euclidean distance)
# A threshold of 1.0 means we consider pairs with cosine similarity >= 0.5
similarity_threshold = 1.0

# Perform the join
neighbors = lsh_model.approxSimilarityJoin(
    normalized_item_vectors,
    normalized_item_vectors,
    threshold=similarity_threshold, 
    distCol="distance"
).filter(col("datasetA.movieId") != col("datasetB.movieId")) # Exclude self-pairs

# Calculate cosine similarity and select relevant columns
neighbors_cosine = neighbors.withColumn(
    "cosine_sim",
    1 - (col("distance") ** 2) / 2
).select(
    col("datasetA.movieId").alias("i_mv"),
    col("datasetB.movieId").alias("j_mv"),
    "cosine_sim"
)

# Make pairs symmetric: (i, j) and (j, i)
# Although approxSimilarityJoin can be non-symmetric, we make it symmetric
# by creating reverse pairs and taking distinct pairs or grouping.
# A simpler approach is to union and then use these pairs in the next step.
similarities = neighbors_cosine.union(
    neighbors_cosine.selectExpr("j_mv as i_mv", "i_mv as j_mv", "cosine_sim")
)

similarities.cache() # Cache for predictions

print("Similar Item Pairs (sample):")
similarities.show(10, truncate=False)

Similar Item Pairs (sample):


+----+-----+------------------+
|i_mv|j_mv |cosine_sim        |
+----+-----+------------------+
|12  |39400|0.5139088171029906|
|27  |191  |0.5587497991102406|
|30  |3475 |0.5437272478270369|
|30  |1176 |0.5970814340265322|
|30  |649  |0.6963106238227913|
|38  |78034|0.6666666666666667|
|38  |4409 |0.506171068243531 |
|43  |8809 |0.5254364539073235|
|77  |2902 |0.6396021490668313|
|92  |25870|0.5132649025747364|
+----+-----+------------------+
only showing top 10 rows



### 3.4 Busca por Itens Similares

Utilizamos o método `approxSimilarityJoin` do LSH para encontrar pares de itens com distância Euclidiana abaixo de um limiar. Convertendo essa distância para similaridade cosseno, obtemos os pares de itens mais similares para recomendações.

#4. Prediction

To predict the rating for a user *u* on a movie *i*, we use the formula:

$$ \hat{r}_{ui} = \frac{\sum_{j \in N(i)} s_{ij} \cdot r_{uj}}{\sum_{j \in N(i)} |s_{ij}|} $$

Where:
* $N(i)$ is the set of neighbors of movie *i*.
* $s_{ij}$ is the similarity between movie *i* and movie *j*.
* $r_{uj}$ is the rating given by user *u* to movie *j*.

We achieve this by joining the `test_data` with `similarities` and then with `train_data`.


In [14]:
# Join test data with similarities to find neighbors for target movies
test_neighbors = test.alias("t") \
    .join(similarities.alias("s"), col("t.movieId") == col("s.i_mv"))

# Join with train data to get user ratings for neighbor movies
test_with_ratings = test_neighbors \
    .join(ratings.alias("r"), (col("t.userId") == col("r.userId")) & (col("s.j_mv") == col("r.movieId"))) \
    .select(
        col("t.userId"),
        col("t.movieId").alias("target_movie"),
        col("s.j_mv").alias("neighbor_movie"),
        col("s.cosine_sim"),
        col("r.rating").alias("neighbor_rating")
    )

print("Test Data with Neighbor Ratings (sample):")
test_with_ratings.show(5, truncate=False)

Test Data with Neighbor Ratings (sample):
+------+------------+--------------+------------------+---------------+
|userId|target_movie|neighbor_movie|cosine_sim        |neighbor_rating|
+------+------------+--------------+------------------+---------------+
|525   |260         |1210          |0.6824355345941335|4.0            |
|477   |260         |1210          |0.6824355345941335|4.5            |
|453   |260         |1210          |0.6824355345941335|3.0            |
|380   |260         |1210          |0.6824355345941335|5.0            |
|337   |260         |1210          |0.6824355345941335|5.0            |
+------+------------+--------------+------------------+---------------+
only showing top 5 rows



## 4. Predição das Avaliações

Para prever a nota que um usuário daria a um filme, buscamos os vizinhos do item alvo e as avaliações do usuário nesses vizinhos. O código a seguir realiza os joins necessários para obter essas informações.

Now, we calculate the weighted average. We add a small epsilon or use `F.when` to avoid division by zero if the sum of similarities is zero.


In [15]:
# Calculate weighted sum and sum of weights
weighted_sums = test_with_ratings.groupBy("userId", "target_movie").agg(
    sql_sum(col("cosine_sim") * col("neighbor_rating")).alias("weighted_rating_sum"),
    sql_sum(F.abs(col("cosine_sim"))).alias("similarity_sum") # Use absolute for the denominator
)

# Predict ratings, handling potential division by zero
predictions = weighted_sums.withColumn(
    "pred_rating",
    F.when(
        col("similarity_sum") > 0,
        col("weighted_rating_sum") / col("similarity_sum")
    ).otherwise(None) 
).filter(col("pred_rating").isNotNull()) \
 .select("userId", "target_movie", "pred_rating")

print("Predicted Ratings (sample):")
predictions.show(5, truncate=False)


Predicted Ratings (sample):


+------+------------+------------------+
|userId|target_movie|pred_rating       |
+------+------------+------------------+
|480   |3793        |3.9043522855700528|
|599   |7193        |2.2990280109681414|
|347   |480         |3.6339209916483215|
|593   |4306        |3.0803749143352657|
|599   |6539        |3.2462228492111707|
+------+------------+------------------+
only showing top 5 rows



### Cálculo da Média Ponderada das Avaliações

Calculamos a média ponderada das avaliações dos vizinhos, utilizando as similaridades como pesos. Isso resulta na predição da nota para cada usuário-filme no conjunto de teste.

We evaluate the predictions against the actual ratings in the `test_data` using **Root Mean Squared Error (RMSE)** and **Mean Absolute Error (MAE)**.


In [16]:
# Join predictions with actual ratings from the test set
final_results = predictions.alias("p") \
    .join(test.alias("t"), (col("p.userId") == col("t.userId")) & (col("p.target_movie") == col("t.movieId"))) \
    .select(
        col("p.userId"),
        col("p.target_movie"),
        col("p.pred_rating"),
        col("t.rating").alias("actual_rating")
    )

# Ensure no null predictions are passed to the evaluator
final_results_filtered = final_results.filter(col("pred_rating").isNotNull())
final_results_filtered.cache()

print("Final Results with Predictions and Actuals (sample):")
final_results_filtered.show(10, truncate=False)


Final Results with Predictions and Actuals (sample):


+------+------------+------------------+-------------+
|userId|target_movie|pred_rating       |actual_rating|
+------+------------+------------------+-------------+
|599   |6811        |1.961901830838415 |3.0          |
|95    |6934        |4.0               |4.0          |
|574   |296         |4.646079373154682 |3.0          |
|452   |2683        |5.0               |5.0          |
|298   |296         |3.843319662443349 |4.5          |
|387   |1527        |3.394992861102659 |3.5          |
|219   |733         |4.5               |3.5          |
|307   |2           |3.0               |2.5          |
|610   |76251       |3.2955652596543876|3.5          |
|489   |1035        |4.5               |3.5          |
+------+------------+------------------+-------------+
only showing top 10 rows



## Avaliação das Predições

Comparamos as predições com as avaliações reais do conjunto de teste utilizando as métricas RMSE (Root Mean Squared Error) e MAE (Mean Absolute Error), que quantificam o erro das recomendações.

In [17]:
# calculate RMSE
evaluator = RegressionEvaluator(
    labelCol="actual_rating",
    predictionCol="pred_rating",
    metricName="rmse"
)
rmse = evaluator.evaluate(final_results_filtered)
print(f"Root Mean Squared Error (RMSE): {rmse}")

#calculate MAE
mae_evaluator = RegressionEvaluator(
    labelCol="actual_rating",
    predictionCol="pred_rating",
    metricName="mae"
)
mae = mae_evaluator.evaluate(final_results_filtered)
print(f"Mean Absolute Error (MAE): {mae}")

Root Mean Squared Error (RMSE): 0.9127885306396452
Mean Absolute Error (MAE): 0.6628536445930843


In [18]:
# Tempo total de execução
end_time = time.time()
execution_time = round(end_time - start_time, 2)

summary = pd.DataFrame([{
    'Dataset':  data_sel,
    'Train Size': ratings.count(),
    'Test Size': test.count(),
    'Similarity': 'Cosine (approximated with LSH)',
    'Similarity Threshold': similarity_threshold,
    'Num Hash Tables': num_hash_tables,
    'Bucket Length': bucket_length,
    'RMSE': rmse,
    'MAE': mae,
    'Execution Time (s)': execution_time
}])

# Salvar o resumo
output_path = './output/item_item_cf_summary.csv'
if os.path.exists(output_path):
    summary.to_csv(output_path, mode='a', header=False, index=False)
else:
    summary.to_csv(output_path, index=False)

print("Resultados salvos em './output/'")


Resultados salvos em './output/'


## Resumo e Salvamento dos Resultados

O código abaixo resume as principais métricas do experimento (tamanho dos conjuntos, parâmetros, erros e tempo de execução) e salva os resultados em um arquivo CSV para análise posterior.

In [19]:
spark.stop()

## Encerramento da Sessão Spark

Por fim, encerramos a sessão Spark para liberar os recursos computacionais utilizados durante o processamento.